In [2]:
import pandas as pd

In [8]:
GENES = pd.read_csv('/home/svanhmic/workspace/data/test/gene_gene_associations_50k.csv',
                    usecols=['OFFICIAL_SYMBOL_A'],
                    low_memory=True,
                    iterator=True,
                    chunksize=1000)


In [9]:
# Concatenate chunks into list & convert to DataFrame
GENES_DF = pd.DataFrame(pd.concat(list(GENES), ignore_index=True))

# Remove duplicates
GENES_DF_CLEAN = GENES_DF.drop_duplicates(keep='first')

# Name Columns
GENES_DF_CLEAN.columns = ['id']

In [13]:
VERTICES = sqlCtx.createDataFrame(GENES_DF_CLEAN)
VERTICES.show()

+------+
|    id|
+------+
|MAP2K4|
|  MYPN|
| ACVR1|
| GATA2|
|  RPA2|
|  ARF1|
|  ARF3|
|  XRN1|
|   APP|
| APLP1|
|CITED2|
| EP300|
|  APOB|
| ARRB2|
| CSF1R|
|PRRC2A|
|  LSM1|
|SLC4A1|
|  BCL3|
| ADRB1|
+------+
only showing top 20 rows

